In [45]:
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime

In [46]:
#load the filtered data 
#please change the directory according to where you stored the file 
df = pd.read_csv("L:\\ML-Assignment\\training_data.csv")


C:\Users\CHEN KANG\AppData\Local\Temp\ipykernel_23876\2607241718.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("L:\\ML-Assignment\\training_data.csv")


In [47]:
df.astype({"joined_text" : "string" , "class" : "string" })

,joined_text,class
0,is upset that he can not update his Facebook b...,0
1,I dived many times for the ball Managed to sav...,0
2,my whole body feels itchy and like its on fire,0
3,no its not behaving at all i am mad why am i h...,0
4,not the whole crew,0
...,...,...
1828592,I did something today I went sledding with my ...,non-suicide
1828593,If you do not like rock then your not going to...,non-suicide
1828594,You how you can tell i have so many friends an...,non-suicide
1828595,pee probably tastes like salty tea can someone...,non-suicide


In [48]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9479647435636171849
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6713745408
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14611368216273913155
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:06:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


In [49]:
#split the data into 2 classes according to the sentiment levels 
splits = list(df.groupby("class"))


negative = splits[0][1]
positive1 = splits[1][1]
positive2 = splits[2][1]
suicidal = splits[4][1]

positive = pd.concat([positive1, positive2], axis = 0)



In [50]:
positive.sample(frac = 1)

,joined_text,class
914149,good topics and ciggarettes,4
1172994,Having lovely time in the sun in a grave yard ...,4
1570338,quotNew DLNA Stackquot is up amp running Suppo...,4
1099451,Let me know what the Pilot 78Gs are like And i...,4
858329,Welcome to Twitter A great start is finding am...,4
...,...,...
1246548,ah cool bro hey still have the xbox video,4
985470,Mary is a fucking ownage singer,4
841709,man i need some stress to work good i can see ...,4
1115089,Car is now cleaned I can actually ferry passen...,4


In [51]:
#randomly select 50000 features from each class 
import random as rand

number = rand.randint(0,750000)
new_positive = positive[number : number + 50000]

number = rand.randint(0,750000)
new_negative = negative[number: number + 50000]

number = rand.randint(0,75000)
new_suicidal = suicidal[number: number+ 50000]

In [52]:
#concat 3 classes into 1 dataframe
df_concat = pd.concat([new_positive, new_negative, new_suicidal], axis = 0)

#shuffle the sequence of the dataframe
df_concat = df_concat.sample(frac = 1)

In [53]:
df_concat

,joined_text,class
1423825,I was at the Zoo from 10noon but only in tweet...,4
1434258,First REALLY good day all weekand it was spent...,4
1640441,Why should not i is not it all just painI do n...,suicide
1408048,Good morning got a few errands to run C you so...,4
1636701,I Am too weak for this worldIm never going to ...,suicide
...,...,...
467694,can not sleepkeep coughin amp can not breath d...,0
1428949,i love the college world series,4
1628559,Putting my plan in motionIve been depressed fo...,suicide
1434623,Awww i am glad 2 sweet baby girl,4


In [54]:
#Split dataframes into random train and test subsets, the method returns lists
#default ratio in splitting the training and testing set is 3:1 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_concat["joined_text"], df_concat["class"], random_state = 0)

In [55]:
#convert the lists into numpy arrays of type string 
x_train = np.asarray(x_train.to_frame().to_numpy(dtype=np.string_).astype(np.string_))
x_test = np.asarray(x_test.to_frame().to_numpy(dtype=np.string_).astype(np.string_))
y_train = np.asarray(y_train.to_frame().to_numpy(dtype=np.string_).astype(np.string_))
y_test = np.asarray(y_test.to_frame().to_numpy(dtype=np.string_).astype(np.string_))


#create labels in the form of a 2 dimensional array to split the labels into 2 distinct classes 
def create_label(y):
    counter = 0
    result = np.array([[]])
    if y[0] == np.string_("0"):
        result = np.array([[1,0,0]])
    elif y[0] == np.string_("4"):
        result = np.array([[0,1,0]])
    elif y[0] == np.string_("suicide"):
        result = np.array([[0,0,1]])
    
    for i in y: 
        if counter > 0:
            if i == np.string_("0"):
                result = np.append(result, np.array([[1,0,0]]), axis = 0)
            elif i == np.string_("4"):
                result = np.append(result, np.array([[0,1,0]]), axis = 0)
            elif i == np.string_("suicide"):
                result = np.append(result, np.array([[0,0,1]]), axis = 0)
        counter+=1
    

    return result


#create labels from the y_train and y_test 
new_y_train = create_label(y_train)   
new_y_test = create_label(y_test)

                

In [40]:
x_test.shape

(37500, 1)

In [41]:
print(new_y_train[30])
print(y_train[30])

[1 0 0]
[b'0']


In [59]:
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, load_model 
from tensorflow.keras.layers import Dense, TextVectorization, LSTM, Dropout
from sklearn.metrics import accuracy_score

In [62]:
#create our machine learning model 
model = Sequential(name = "Nice_model")
model.add(Input(shape=(1,), dtype = tf.string))

#text vectorization is needed to convert the texts into integers 
vectorize_layer = TextVectorization(split = "whitespace", 
                                    ngrams=(1,2),
                                    output_mode = "tf_idf")

vectorize_layer.adapt(np.asarray(df_concat["joined_text"].to_numpy(dtype=np.string_)).astype(np.string_))


#add the layers into our models 
model.add(vectorize_layer)

model.add(Dropout(0.2, input_shape=(128,)))
model.add(Dense(units = 128, activation = "relu", name = "L1"))
model.add(Dropout(0.2))
model.add(Dense(units = 128, activation = "relu", name = "L2"))
model.add(Dropout(0.2))
model.add(Dense(units = 64, activation = "relu", name = "L3"))
model.add(Dropout(0.2))
model.add(Dense(units = 3, activation = "sigmoid", name = "Output"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics = "accuracy")

print("OK")

OK


In [63]:
model.summary()

Model: "Nice_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_4 (TextV  (None, 1555836)          1         
 ectorization)                                                   
                                                                 
 dropout_3 (Dropout)         (None, 1555836)           0         
                                                                 
 L1 (Dense)                  (None, 128)               199147136 
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 L2 (Dense)                  (None, 128)               16512     
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                        

In [ ]:
#display the score of our model 
score = model.evaluate(x_test, new_y_test)

print(score[0])
print(score[1])

In [65]:
#start the training here 
#please change the directory according to where you wish to store the logs 
log_dir = "L:\\ML-Assignment\\Logs\\Logs" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

history = model.fit(x_train, new_y_train,
         epochs=4,
         batch_size=64,
         callbacks=[tensorboard_callback],
         verbose = 1,
         validation_data = (x_test, new_y_test))


Epoch 1/4
1758/1758 [==============================] - 893s 508ms/step - loss: 0.2125 - accuracy: 0.8652 - val_loss: 0.2225 - val_accuracy: 0.8543
Epoch 2/4
1758/1758 [==============================] - 888s 505ms/step - loss: 0.0661 - accuracy: 0.9619 - val_loss: 0.2805 - val_accuracy: 0.8417
Epoch 3/4
1758/1758 [==============================] - 891s 507ms/step - loss: 0.0282 - accuracy: 0.9867 - val_loss: 0.3683 - val_accuracy: 0.8410
Epoch 4/4
1758/1758 [==============================] - 890s 507ms/step - loss: 0.0151 - accuracy: 0.9919 - val_loss: 0.3850 - val_accuracy: 0.8393


In [67]:
%reload_ext tensorboard
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 19612), started 22:07:22 ago. (Use '!kill 19612' to kill it.)

In [ ]:
print(y_train)
print(new_y_train)

In [68]:
#save the model, remember to change the model number in the file name 
#please change the directory according to where you wish to store the model 
model_dir = "L:\\ML-Assignment\\Model\\Model_07"
model.save(model_dir)

INFO:tensorflow:Assets written to: L:\ML-Assignment\Model\Model_07\assets


In [69]:
#delete the model after it is saved 
del model

In [70]:
#load the model 
model = load_model(model_dir)

print("Loaded")

Loaded


In [71]:
#clean the input like how we cleaned the data for the model 
import contractions
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np 


def predict(model, input, stopword = stopwords):
    input = input.lower()
    input = re.sub("@[^\s]+", '', input)
    input = re.sub(r'[^\w\s]', '', input)
    input = input.split(" ")

    input = [x.encode("ascii","ignore").decode() for x in input]
    input = [contractions.fix(x) for x in input]
    input = ' '.join(map(str, input))
    input = word_tokenize(input, language = 'english')
    #input = [x for x in input if x not in stopwords]
    input = [x for x in input if not x.isdigit()]
    input = ' '.join(map(str, input))
    filtered_text = input
    
    x_input = np.array([input], dtype = np.string_)
    y = model.predict(x_input)
    result = ""
    
    if y[0][0] == y[0].max():
        result = "High probability of depression"
    elif y[0][1] == y[0].max():
        result = "Low probabiltity of depression"
    elif y[0][2] == y[0].max():
        result = "Suicidal thoughts"
        
    return (result, y, filtered_text)

#return a list wit


In [74]:
#test the model with inputs 
while True: 
    x = input("Enter Text: ")
    
    if x == "quit" or x == "exit":
        break
    
    y = predict(model, x)
    print("Filtered Text: " + y[2])
    print("Model Result: " + y[0])
    print("Probability of depression: ", "{:.4f}%".format(y[1][0][0] * 100))
    print("No depression: ", "{:.4f}%".format(y[1][0][1] * 100) )
    print("Suicidal thought: ", "{:.4f}%".format(y[1][0][2] * 100) )



    
print("Completed.")

Enter Text: Everything I do is a drag, I don't have much ambition or enjoyment in life and everything I do, having no friends or a relationship just hurts, feels like I have no reason to continue apart from doing things I'm already bored with and just living for the sake of it, I don't have much to keep me going, apart from false hopes and fantasies of my life getting better, my life just feels like a long process I can't wait to get out of, because I honestly believe that death is a better state than living as myself, I can't stand living with myself and how worthless I am, it appears that suicide isn't a frightening thought for me, but living a life I can't stand is.
1/1 [==============================] - 1s 551ms/step
Filtered Text: everything i do is a drag i do not have much ambition or enjoyment in life and everything i do having no friends or a relationship just hurts feels like i have no reason to continue apart from doing things i am already bored with and just living for the 

Enter Text: im gonna inhale some carbon dioxide and die peacefully
1/1 [==============================] - 0s 40ms/step
Filtered Text: i am going to inhale some carbon dioxide and die peacefully
Model Result: Suicidal thoughts
Probability of depression:  0.0001%
No depression:  0.0000%
Suicidal thought:  99.9998%
Enter Text: im going to inhale some carbon monoxid
1/1 [==============================] - 0s 32ms/step
Filtered Text: i am going to inhale some carbon monoxid
Model Result: Low probabiltity of depression
Probability of depression:  13.1248%
No depression:  69.7144%
Suicidal thought:  27.0477%
Enter Text: i am going to inhale some carbon dioxide
1/1 [==============================] - 0s 40ms/step
Filtered Text: i am going to inhale some carbon dioxide
Model Result: Suicidal thoughts
Probability of depression:  0.3494%
No depression:  0.3169%
Suicidal thought:  99.1634%
Enter Text: i am going to jump off a bridge
1/1 [==============================] - 0s 40ms/step
Filtered Text: 

1/1 [==============================] - 0s 27ms/step
Filtered Text: i do not want to exist
Model Result: High probability of depression
Probability of depression:  88.7701%
No depression:  6.1068%
Suicidal thought:  3.5494%
Enter Text: let me go 
1/1 [==============================] - 0s 25ms/step
Filtered Text: let me go
Model Result: High probability of depression
Probability of depression:  69.9971%
No depression:  29.5008%
Suicidal thought:  4.2755%
Enter Text: let me go peacefully 
1/1 [==============================] - 0s 33ms/step
Filtered Text: let me go peacefully
Model Result: Suicidal thoughts
Probability of depression:  27.7799%
No depression:  39.3395%
Suicidal thought:  44.7810%
Enter Text: peacefully
1/1 [==============================] - 0s 25ms/step
Filtered Text: peacefully
Model Result: Low probabiltity of depression
Probability of depression:  22.2179%
No depression:  76.7956%
Suicidal thought:  4.4709%
Enter Text: dying peacefully
1/1 [==============================

KeyboardInterrupt: Interrupted by user